### Load environment variables

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

os.environ["OTEL_SDK_DISABLED"] = "true"

### Initialize LLM

In [4]:
from crewai import LLM

llm = LLM(
    model = "gpt-4o-mini",
    stop = ["END"],
    seed = 42
)

### Define the Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai.tools import tool, BaseTool

search_tool = SerperDevTool(n_results=3)
scrape_tool = ScrapeWebsiteTool()

In [7]:
# Method-1 to create a custom tool
class CalculatorTool(BaseTool):
    name: str = "Calculator Tool"
    description: str = (
        "Useful for performing mathematical calculations, "
        "such as addition, subtraction, multiplication, division, etc. "
        "The input to this tool should be a mathematical expression as a string. "
        "For example, `200*7` or `5000/2*10`."
    )
    
    def _run(self, operation: str) -> str:
        try:
            return eval(operation)
        except Exception as e:
            return f"Error in calculation: {str(e)}"


# Method-2 to create a custom tool
class MyCalculatorTool():
    @tool("Calculator Tool")
    def calculate(operation):
        """Useful to perform any mathematical calculations, 
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical 
        expression.`
        """
        return eval(operation)

### Create Agents

In [8]:
from crewai import Agent

In [9]:
city_selector_agent = Agent(
    role = "City Selection Expert",
    goal = "Select the best city based on weather, season, and prices",
    backstory = "An expert in analyzing travel data to pick ideal destinations",
    verbose = True,
    tools = [search_tool, scrape_tool],
    llm = llm,
    # allow_delegation = False,
)


local_expert_agent = Agent(
    role = "Local Expert at this city",
    goal = "Provide the BEST insights about the selected city",
    backstory = "A knowledgeable local guide with extensive information about the city, it's attractions and customs",
    verbose = True,
    tools = [search_tool, scrape_tool],
    llm = llm,
    # allow_delegation = False,
)


travel_concierge_agent = Agent(
    role = "Amazing Travel Concierge",
    goal = "Create the most amazing travel itineraries with budget and packing suggestions for the city",
    backstory = "Specialist in travel planning and logistics with decades of experience",
    verbose = True,
    tools = [search_tool, scrape_tool, MyCalculatorTool.calculate],
    llm = llm,
    # allow_delegation = False,
)


accommodation_agent = Agent(
    role = "Accommodation Specialist",
    goal = "Find the best hotels/Airbnbs close to places of interest within budget",
    backstory = "An expert in finding perfect accommodations for travelers",
    verbose = True,
    tools = [search_tool, scrape_tool, MyCalculatorTool.calculate],
    llm = llm,
    # allow_delegation = False,
)


food_expert_agent = Agent(
    role = "Culinary Expert",
    goal = "Recommend the best food places catering to traveler preferences",
    backstory = "A food critic and local cuisine enthusiast",
    verbose = True,
    tools = [search_tool, scrape_tool],
    llm = llm,
    # allow_delegation = False,
)


interest_specialist_agent = Agent(
    role = "Interest and Activity Specialist",
    goal = "Suggest activities and places aligned with traveler interests",
    backstory = "An expert in curating personalized experiences",
    verbose = True,
    tools = [search_tool, scrape_tool],
    llm = llm,
    # allow_delegation = False,
)

### Define the Tasks

In [10]:
from crewai import Task
from textwrap import dedent

In [11]:
city_identify_task = Task(
    description = dedent("""
            Analyze and select the best city for the trip based
            on specific criteria such as weather patterns, seasonal
            events, and travel costs. This task involves comparing
            multiple cities, considering factors like current weather
            conditions, upcoming cultural or seasonal events, and
            overall travel expenses.

            Your final answer must be a detailed
            report on the chosen city, and everything you found out
            about it, including the actual flight costs, weather
            forecast and attractions.

            Traveling from: {origin}
            City Options: {cities}
            Trip Date: {range}
            Traveler Interests: {interests}
          """),
            expected_output = "A detailed report on the chosen city with flight costs, weather forecast, and attractions.",
            agent = city_selector_agent)

In [12]:
gather_information = Task(
    description = dedent("""
            As a local expert on this city you must compile an
            in-depth guide for someone traveling there and wanting
            to have THE BEST trip ever!
            Gather information about  key attractions, local customs,
            special events, and daily activity recommendations.
            Find the best spots to go to, the kind of place only a
            local would know.
            This guide should provide a thorough overview of what
            the city has to offer, including hidden gems, cultural
            hotspots, must-visit landmarks, weather forecasts, and
            high level costs.

            The final answer must be a comprehensive city guide,
            rich in cultural insights and practical tips,
            tailored to enhance the travel experience.

            Trip Date: {range}
            Traveling from: {origin}
            Traveler Interests: {interests}
          """),
            expected_output = "A comprehensive city guide with cultural insights and practical tips.",
            agent = local_expert_agent)

In [13]:
plan_task = Task(
    description = dedent("""
            Expand this guide into a full travel
            itinerary for this time {range} with detailed per-day plans, including
            weather forecasts, places to eat, packing suggestions,
            and a budget breakdown.

            You MUST suggest actual places to visit, actual hotels
            to stay and actual restaurants to go to.

            This itinerary should cover all aspects of the trip,
            from arrival to departure, integrating the city guide
            information with practical travel logistics.

            Your final answer MUST be a complete expanded travel plan,
            formatted as markdown, encompassing a daily schedule,
            anticipated weather conditions, recommended clothing and
            items to pack, and a detailed budget, ensuring THE BEST
            TRIP EVER, Be specific and give it a reason why you picked
            # up each place, what make them special! 

            Trip Date: {range}
            Traveling from: {origin}
            Traveler Interests: {interests}
          """),
            expected_output = "A complete 7-day travel plan, formatted as markdown, with a daily schedule and budget.",
            agent = travel_concierge_agent)

In [14]:
accommodation_task = Task(
    description = dedent("""
            Find and recommend the best hotels/Airbnbs close to the main attractions
            and areas of interest. Consider the following:
            1. Proximity to main attractions and areas of interest
            2. User's budget constraints
            3. Amenities and ratings
            4. Availability for the specified date range

            Provide at least 3 options with details including:
            - Name and type (hotel/Airbnb)
            - Location and proximity to attractions
            - Price per night
            - Brief description and key amenities
            - Link or booking information

            Trip Date: {range}
            Budget per night: {accommodation_budget}
            Traveler Interests: {interests}
        """),
    expected_output = "A list of at least 3 accommodation options with detailed information.",
    agent = accommodation_agent
)

In [15]:
food_recommendation_task = Task(
    description = dedent("""
            Recommend the best food places in the city, considering:
            1. Proximity to accommodations and attractions
            2. Local specialties and cuisine types
            3. Traveler's dietary preferences and budget
            4. A mix of famous restaurants and hidden gems

            Provide at least 5 recommendations with:
            - Restaurant name
            - Cuisine type
            - Location
            - Price range
            - Signature dishes
            - Brief description of why it's recommended

            Traveler Interests: {interests}
            Dietary Preferences: {dietary_preferences}
            Food Budget: {food_budget}
        """),
    expected_output = "A list of at least 5 food place recommendations with detailed information.",
    agent = food_expert_agent
)

In [16]:
interest_based_activity_task = Task(
    description = dedent("""
            Suggest activities and places to visit based on the traveler's specific interests.
            Consider:
            1. Traveler's stated interests
            2. Seasonal events or activities
            3. Unique experiences related to the destination
            4. A mix of popular and off-the-beaten-path suggestions

            Provide at least 5 suggestions with:
            - Activity or place name
            - Type of activity
            - Location
            - Estimated time required
            - Cost (if applicable)
            - Why it matches the traveler's interests

            Trip Date: {range}
            Traveler Interests: {interests}
        """),
    expected_output = "A list of at least 5 interest-based activity suggestions with detailed information.",
    agent = interest_specialist_agent
)

### Creating and Executing the Crew

In [17]:
from crewai import Crew, Process

trip_planner_crew = Crew(
    agents = [city_selector_agent, local_expert_agent, travel_concierge_agent,
              accommodation_agent, food_expert_agent, interest_specialist_agent],
    tasks = [city_identify_task, gather_information, accommodation_task,
             food_recommendation_task, interest_based_activity_task, plan_task],
    verbose = True
)

In [18]:
trip_details = {
    'origin': 'Chicago, USA',
    'cities': 'Mumbai, India',
    'range': '10 October 2024 to 20 October 2024',
    'interests': 'Sightseeing, Photography, Games and Adventure Activities',
    'accommodation_budget': '$200-300 per night',
    'food_budget': '$100 per meal',
    'dietary_preferences': 'Vegetarian options'
}

In [19]:
result = trip_planner_crew.kickoff(inputs=trip_details)

 
[2025-03-14 09:36:26][🚀 CREW 'CREW' STARTED, ACA9EF13-E6E4-4B76-A85E-79A60CF4B6C7]: 2025-03-14 09:36:26.295694
 
[2025-03-14 09:36:26][📋 TASK STARTED: 
ANALYZE AND SELECT THE BEST CITY FOR THE TRIP BASED
ON SPECIFIC CRITERIA SUCH AS WEATHER PATTERNS, SEASONAL
EVENTS, AND TRAVEL COSTS. THIS TASK INVOLVES COMPARING
MULTIPLE CITIES, CONSIDERING FACTORS LIKE CURRENT WEATHER
CONDITIONS, UPCOMING CULTURAL OR SEASONAL EVENTS, AND
OVERALL TRAVEL EXPENSES.

YOUR FINAL ANSWER MUST BE A DETAILED
REPORT ON THE CHOSEN CITY, AND EVERYTHING YOU FOUND OUT
ABOUT IT, INCLUDING THE ACTUAL FLIGHT COSTS, WEATHER
FORECAST AND ATTRACTIONS.

TRAVELING FROM: CHICAGO, USA
CITY OPTIONS: MUMBAI, INDIA
TRIP DATE: 10 OCTOBER 2024 TO 20 OCTOBER 2024
TRAVELER INTERESTS: SIGHTSEEING, PHOTOGRAPHY, GAMES AND ADVENTURE ACTIVITIES
]: 2025-03-14 09:36:26.301770
 
[2025-03-14 09:36:26][🤖 AGENT 'CITY SELECTION EXPERT' STARTED TASK]: 2025-03-14 09:36:26.302245
# Agent: City Selection Expert
## Task: 
Analyze and select the 

In [20]:
from IPython.display import Markdown
Markdown(result.raw)

# Mumbai Travel Itinerary: October 10 - October 20, 2024

## Overview
- **Trip Duration:** 10 Days
- **Traveling From:** Chicago, USA
- **Interests:** Sightseeing, Photography, Games, and Adventure Activities
- **Estimated Budget:** $1,500 - $2,000 (including flights, accommodation, food, and entertainment)

---

## Daily Itinerary

### Day 1: Arrival in Mumbai (October 10)
- **Flight:** Arrive at Chhatrapati Shivaji Maharaj International Airport (Recommended Flight: $719 on Expedia)
- **Weather:** High 82°F, Thunderstorms and Rain
- **Accommodation:** The Taj Mahal Palace, $250/night
  - *Why:* Iconic luxury hotel, close to major attractions like the Gateway of India.
- **Afternoon:**
  - **Activity:** Check-in and Freshen up
  - **Lunch:** Dining at **The Lotus Cafe** (approx. $25)
    - *Why:* Specializes in buffets with various vegetarian options in a serene environment.
- **Evening:**
  - **Activity:** Explore **Marine Drive**, perfect for photography at sunset.
  - **Dinner:** **Fiona**, Italian and Indian fusion ($20)
  
---

### Day 2: Cultural Exploration (October 11)
- **Weather:** Overcast with Rain, High 82°F
- **Breakfast:** Hotel Buffet (included)
- **Morning:**
  - **Activity:** Visit **Gateway of India** for iconic photo opportunities.
- **Lunch:** Traditional street food at **Chowpatty Beach** ($10)
- **Afternoon:**
  - **Activity:** Take a ferry to **Elephanta Caves**, visit ancient rock-cut temples ($5 ferry, $2 entrance).
- **Evening:**
  - **Activity:** Visit **Dhobi Ghat** for unique photography.
  - **Dinner:** **Kebabs & Kurries**, vegetarian kebab platter ($25)
  
---

### Day 3: Adventure Activities (October 12)
- **Weather:** Warm and Humid, High 84°F
- **Breakfast:** Local café or hotel (included)
- **Morning:**
  - **Activity:** **Zero Latency India** (VR Gaming Zone), approx. $25.
- **Lunch:** **Namak**, authentic Indian Cuisine ($20)
- **Afternoon:**
  - **Activity:** **Mystery Rooms** escape game, approx. $20.
- **Evening:**
  - **Activity:** Visit **Juhu Beach** for street food ($10) and local vibes.
  
---

### Day 4: Festival Celebration (October 13)
- **Weather:** Possible Rain, High 82°F
- **Breakfast:** Local café 
- **Morning:**
  - **Activity:** Attend **Ganesh Chaturthi celebrations** near local Pandals.
- **Lunch:** **Tuskers**, vegetarian dining, approx. $20
- **Afternoon:**
  - **Activity:** Explore **Colaba Causeway** for handicrafts and local markets.
- **Dinner:** **The Pantry**, healthy vegetarian options ($20)
  
---

### Day 5: Outdoor Adventures (October 14)
- **Weather:** Humid, High 84°F
- **Breakfast:** Hotel
- **Morning:**
  - **Activity:** Early trek at **Kanheri Caves**, entry fees $3.
- **Lunch:** Pack a picnic to enjoy in the park.
- **Evening:**
  - **Activity:** Explore **Chor Bazaar** for unique finds.
  - **Dinner:** **Gajalee**, famous for seafood (approx. $25).
  
---

### Day 6: Relaxing Day (October 15)
- **Weather:** Partly Cloudy, High 82°F
- **Breakfast:** Local café or hotel (included)
- **Morning:**
  - **Activity:** Yoga session at **Marine Drive**.
- **Lunch:** **Prakash Upahaar Kendra**, known for fantastic breakfast options ($10).
- **Afternoon:**
  - **Activity:** Visit **Bandra-Worli Sea Link** for great views.
- **Dinner:** **Café Madras**, casual dining ($15).
  
---

### Day 7: Last Day in Mumbai (October 16)
- **Weather:** Mild and Humid, High 82°F
- **Breakfast:** Hotel
- **Morning:** Last-minute shopping at **Phoenix Marketcity**.
- **Lunch:** **Sukh Sagar** for local vegetarian fare ($15).
- **Afternoon:** Relax at **Hanging Gardens**.
- **Evening:**
  - **Activity:** Farewell dinner at **Ziya**, upscale dining experience ($50).
  
---

## Packing Suggestions
- **Clothing:** Lightweight, breathable clothing; rain jacket; comfortable walking shoes.
- **Essentials:** Sunscreen, insect repellent, reusable water bottle (stay hydrated), camera (for photography).
- **Health Items:** Personal medications and hand sanitizer.

---

## Budget Breakdown
- **Flights:** $719 (round trip)
- **Accommodation:** $2,250 (10 nights at $250/night)
- **Food:** ~$400 (avg. $40/day)
- **Activities & Transport:** ~$400 (includes local transport, attractions)
- **Total Estimated Budget:** ~$3,769

Enjoy your adventure in the vibrant city of Mumbai, filled with epic cultural experiences, rich flavors, and picturesque views!

In [21]:
print(f"Tasks Output: {result.tasks_output}")
print(f"Token Usage: {result.token_usage}")

Tasks Output: [TaskOutput(description='\nAnalyze and select the best city for the trip based\non specific criteria such as weather patterns, seasonal\nevents, and travel costs. This task involves comparing\nmultiple cities, considering factors like current weather\nconditions, upcoming cultural or seasonal events, and\noverall travel expenses.\n\nYour final answer must be a detailed\nreport on the chosen city, and everything you found out\nabout it, including the actual flight costs, weather\nforecast and attractions.\n\nTraveling from: Chicago, USA\nCity Options: Mumbai, India\nTrip Date: 10 October 2024 to 20 October 2024\nTraveler Interests: Sightseeing, Photography, Games and Adventure Activities\n', name=None, expected_output='A detailed report on the chosen city with flight costs, weather forecast, and attractions.', summary='\nAnalyze and select the best city for the trip based\non...', raw="**Detailed Report on Mumbai for Trip from Chicago (October 10 - October 20, 2024)**\n\n*

In [24]:
import pandas as pd

costs = (result.token_usage.prompt_tokens + result.token_usage.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

df_usage_metrics = pd.DataFrame([result.token_usage.model_dump()])
df_usage_metrics

Total costs: $0.0812


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,81166,74985,36864,6181,21
